In [1]:
#Import necesssary libraries

import requests
from urllib.parse import urlencode, urlparse, parse_qsl

import numpy as np
import pandas as pd

In [2]:
#Read the API KEY from the text file
api_file = open("API_key.txt","r")
GOOGLE_API_KEY = api_file.read()
api_file.close()

##### User Inputs (office address & Desired Commute time)

In [3]:
office_address = input("Enter your complete office address: \n") #Get office address as user input


In [4]:
desired_commute_time = int(input("What is your maximum preferred commute time (in mins)? \n"))*60
desired_commute_time

7200

#### Geolocation details of the given office address using Geocode API

In [22]:
data_type = "json"
#def extract_office_info (office_address,data_type='json'):
geocode_base_url = f"https://maps.googleapis.com/maps/api/geocode/{data_type}"
geocode_params = {"address": "NewQuay Docklands VIC 3008, Australia", "key": GOOGLE_API_KEY}
geocode_url_params = urlencode(geocode_params)
geocode_enpoint_url = f"{geocode_base_url}?{geocode_url_params}"
geocode_response = requests.get(geocode_enpoint_url)
geocode_data = geocode_response.json()['results'][0]
if geocode_response.status_code not in range (200,299):
    {}
latlng = {}
try:
    latlng = geocode_data['geometry']['location']
    office_place_id = geocode_data['place_id']

    for component in geocode_data['address_components']:
        if all(t in component['types'] for t in ['locality']):
            office_locality = component['long_name']
        elif all(t in component['types'] for t in ['administrative_area_level_1']):
            office_state = component['short_name']
        elif all(t in component['types'] for t in ['postal_code']):
            office_postal_code = component['long_name']
        else:
            all(t in component['types'] for t in ['Australia'])
            office_country = component['long_name']
except:
    pass
lat,lng = latlng.get("lat"),latlng.get("lng")
search_text = f"{office_locality} {office_state} {office_country}"
    #return lat,lng, office_place_id,office_postalcode,locality

In [23]:
#Parse locations details (Latitute, Longitude, Postal Code and Suburbs details)
print(f' Below are the details for the entered office location:\n\n Address Entered: {office_address} \n Latitude: {lat},Longitute: {lng}\n Place_id: {office_place_id}\n Suburb: {office_locality} \n Postal_code: {office_postal_code} \n')

 Below are the details for the entered office location:

 Address Entered: 833, collins street, docklands, vic 
 Latitude: -37.8143399,Longitute: 144.9427283
 Place_id: ChIJBWcnO0Nd1moRprKCB3ML6HY
 Suburb: Docklands 
 Postal_code: 3008 



## Approach 1: Getting suburbs using "Nearby Search"

In [24]:
nearby_base_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json"
nearby_params = {
    "key" : GOOGLE_API_KEY,
    "location" : f"{lat},{lng}",
    "type": "locality",
    #"regions" : "administrative_area_level_2",
    #"radius" : 40000,
    "rankby" : "distance",  
    }
nearby_url_params = urlencode(nearby_params)
nearby_endpoint_url = f"{nearby_base_url}?{nearby_url_params}"
nearby_response = requests.get(nearby_endpoint_url)
nearby_data = nearby_response.json()["results"]

In [25]:
nearby_data

[{'geometry': {'location': {'lat': -37.8190118, 'lng': 144.9465764},
   'viewport': {'northeast': {'lat': -37.8092488479762,
     'lng': 144.9559920049019},
    'southwest': {'lat': -37.82702282857287, 'lng': 144.9200560072265}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
  'icon_background_color': '#7B9EB0',
  'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',
  'name': 'Docklands',
  'photos': [{'height': 2592,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/110723516400199105649">Harsha Dunuji</a>'],
    'photo_reference': 'AZose0nEYAnNyUmkmtxzhFMjKWdapbrISzNm3jNBl74a1SvyeZh1uzojeXyPsrgGwFzolA7NgPv7kEvrzwArI88SyhcNYEJr4twjsVG0_Uh-YjWqILWBA5BAhNNC6X3CUYuxORa2I-zJQdVPoQk5V80c0bxrBzwCr9yZSqI-y4eVDV0yKsAP',
    'width': 4608}],
  'place_id': 'ChIJb9QWRmdd1moR8P2MIXVWBAU',
  'reference': 'ChIJb9QWRmdd1moR8P2MIXVWBAU',
  'scope': 'GOOGLE',
  'types': ['locality', 'political'],


#### List of suburbs with commute time less than the desired commute time (DistanceMatrix API)

In [26]:
nearby_neighborhoods = []
for sub in nearby_data:
    location_id = sub['place_id']
    location = sub['geometry']['location']
    location_lat = location['lat']
    location_lng = location['lng']
    
    distance_base_url = f"https://maps.googleapis.com/maps/api/distancematrix/json"
    distance_params = {
    "key" : GOOGLE_API_KEY,
    "origins" : f"place_id:{office_place_id}",
    "destinations" : f"place_id:{location_id}",
    "mode" : "transit",
    #"transit_mode" : "rail|bus",
    #"type" : "locality"
    }
    distance_url_params = urlencode(distance_params)
    distance_endpoint_url = f"{distance_base_url}?{distance_url_params}"
    distance_response = requests.get(distance_endpoint_url)
    distance_data = distance_response.json()
    commute_duration = distance_data['rows'][0]['elements'][0]['duration']['value']

    if commute_duration <= desired_commute_time:
        nearby_neighborhoods.append(distance_data)
        print(f"{distance_data['destination_addresses'][0]} | Commute duration (in seconds): {commute_duration}")

Docklands VIC 3008, Australia | Commute duration (in seconds): 558
Melbourne VIC, Australia | Commute duration (in seconds): 1403


In [ ]:
# nearby_neighborhoods

## Approach 2: Getting suburbs using "Find Place"

In [12]:
input_text =  f"suburb near {search_text}"
findplace_base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
findplace_params = {
    "key" : GOOGLE_API_KEY,
    "input" : input_text,
    "inputtype" : "textquery",
    "fields" : "place_id,name,geometry,type",
    "locationbias" : f'point:{lat},{lng}',
}
findplace_url_params = urlencode(findplace_params)
findplace_endpoint_url = f'{findplace_base_url}?{findplace_url_params}'
findplace_response = requests.get(findplace_endpoint_url)
findplace_data = findplace_response.json()['candidates']

In [27]:
findplace_data

[{'geometry': {'location': {'lat': -37.8143399, 'lng': 144.9427283},
   'viewport': {'northeast': {'lat': -37.8055245, 'lng': 144.9587357},
    'southwest': {'lat': -37.8231542, 'lng': 144.9267209}}},
  'name': 'NewQuay',
  'place_id': 'ChIJBWcnO0Nd1moRprKCB3ML6HY',
  'types': ['neighborhood', 'political']},
 {'geometry': {'location': {'lat': -37.8123652, 'lng': 144.9623382},
   'viewport': {'northeast': {'lat': -37.79944589999999, 'lng': 144.9748669},
    'southwest': {'lat': -37.8230788, 'lng': 144.951435}}},
  'name': 'CBD',
  'place_id': 'ChIJxXF388pC1moRYe3cJE1mToM',
  'types': ['neighborhood', 'political']},
 {'geometry': {'location': {'lat': -37.8190118, 'lng': 144.9465764},
   'viewport': {'northeast': {'lat': -37.8092488, 'lng': 144.955992},
    'southwest': {'lat': -37.8270228, 'lng': 144.920056}}},
  'name': 'Docklands',
  'place_id': 'ChIJb9QWRmdd1moR8P2MIXVWBAU',
  'types': ['locality', 'political']},
 {'geometry': {'location': {'lat': -37.8125536, 'lng': 144.9369982},
   

In [29]:
nearby_suburbs = []
for suburb in findplace_data:
    is_valid = any(t in ['neighborhood','locality'] for t in suburb["types"])
    if (suburb["place_id"]!=office_place_id) and (is_valid):
        suburb_name = suburb["name"]
        suburb_place_id = suburb["place_id"]
        suburb_type = suburb["types"]
        suburb_location = suburb["geometry"]["location"]
        suburb_lat = suburb_location["lat"]
        suburb_lng = suburb_location["lng"]
        
        distance_base_url = f"https://maps.googleapis.com/maps/api/distancematrix/json"
        distance_params = {
        "key" : GOOGLE_API_KEY,
        "origins" : f"place_id:{office_place_id}",
        "destinations" : f"place_id:{suburb_place_id}",
        "mode" : "transit",
        #"transit_mode" : "rail|bus",
        #"type" : "locality"
        }
        distance_url_params = urlencode(distance_params)
        distance_endpoint_url = f"{distance_base_url}?{distance_url_params}"
        distance_response = requests.get(distance_endpoint_url)
        distance_data = distance_response.json()
        commute_time = distance_data['rows'][0]['elements'][0]['duration']['value']

        if commute_time <= desired_commute_time:
            nearby_suburbs.append({
                                "Suburb Name" : suburb_name,
                                "Commute Time (mins)" : np.round(commute_time/60,2)
                                })
            print(f"{distance_data['destination_addresses'][0]} | Commute duration (in mins): {commute_time/60 :.2f}")

CBD, Melbourne VIC 3000, Australia | Commute duration (in mins): 26.85
Docklands VIC 3008, Australia | Commute duration (in mins): 9.30
North Melbourne VIC 3051, Australia | Commute duration (in mins): 28.17
East End, Melbourne VIC 3000, Australia | Commute duration (in mins): 29.05
South Melbourne VIC 3205, Australia | Commute duration (in mins): 40.93
Coode Island, West Melbourne VIC 3003, Australia | Commute duration (in mins): 52.22
Jolimont, East Melbourne VIC 3002, Australia | Commute duration (in mins): 33.88
Southbank VIC 3006, Australia | Commute duration (in mins): 36.85
Seddon VIC 3011, Australia | Commute duration (in mins): 31.10
Carlton VIC 3053, Australia | Commute duration (in mins): 41.35
South Wharf VIC 3006, Australia | Commute duration (in mins): 21.37


In [31]:
pd.DataFrame(nearby_suburbs).sort_values(by="Commute Time (mins)",ascending=True)

,Suburb Name,Commute Time (mins)
1,Docklands,9.30
10,South Wharf,21.37
0,CBD,26.85
2,North Melbourne,28.17
3,East End,29.05
8,Seddon,31.10
6,Jolimont,33.88
7,Southbank,36.85
4,South Melbourne,40.93
9,Carlton,41.35
